In [2]:
# 영상을 받아들일 카메라 인스턴스 생성 

from jetbot import Camera 
from jetbot import bgr8_to_jpeg
import PID

camera = Camera.instance(width=720, height=720)  # 720x720 픽셀 이미지 사용

In [3]:
# 실행에 필요한 전역변수 생성
global body_x, body_y, body_w, body_h
body_x = body_y = body_w = body_h = 0

global target_valuex
target_valuex = 0

global target_valuey
target_valuey = 0

In [4]:
# 키메라 팬/틸트 제어를 위해 PID 인스턴스 생성
xservo_pid = PID.PositionalPID(1.9,0.3,0.35)  # (P, I, D)
yservo_pid = PID.PositionalPID(1.5,0.2,0.3)  # (P, I, D)


In [5]:
# 서보모터 인스턴스 생성
from servoserial import ServoSerial

servo_device = ServoSerial()

serial Open!


In [6]:
# 카메라 프리뷰 위젯 생성
import traitlets
import ipywidgets.widgets as widget
from IPython.display import display

body_image = widget.Image(format='jpeg',width = 300, height = 300)
display(body_image)

Image(value=b'', format='jpeg', height='300', width='300')

In [7]:
# HAAR Cascade CLassifier 읽기
import cv2
body_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# body 인식 및 팬/틸트 모듈제어로 추적하기
while True:
    frame = camera.value
    frame= cv2.resize(frame,(300,300))  # 이미지를 학습 데이터와 동일한 크기로 변환
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  # 흑백 이미지로 변환하여 검출에 사용
    
    bodies = body_cascade.detectMultiScale(gray)  # 이미지 내에서 얼굴을 검출
    
    if len(bodies) > 0:  # 검출된 내용이 있으면        
        (body_x, body_y, body_w, body_h) = bodies[0]  # 좌표와 크기
        
        # 검출 영역을 사각 박스로 표시합니다.
        cv2.rectangle(frame, (body_x, body_y),(body_x+body_w, body_y+body_h), (0,255,123), 5)
        
        # PID(Proportion Integration Differentiation) 알고리즘 적용에 의한 카메라 방향 제어
        
        # x축 오프셋을 PID 컨트롤에 입력
        
        xservo_pid.SystemOutput = body_x + body_h/2  # x축 중심점 모듈에서 초기값은 0.0
        xservo_pid.SetStepSignal(150)  # Set the PID controller parameters
        xservo_pid.SetInertiaTime(0.01,0.006)  # Set the first-order inertial link system, where InertiaTime is the inertia time constant
        target_valuex = int(2048+xservo_pid.SystemOutput)
        
        # y축 오프셋을 PID 컨트롤에 입력
        yservo_pid.SystemOutput = body_y + body_w/2  # y축 중심점
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01,0.006)
        target_valuey = int(2048+yservo_pid.SystemOutput)
        
        # PID 조정 위치로 카메라 팬/틸트 서조를 움직임 (팬 = 1, 틸트 = 2)
        servo_device.Servo_serial_double_control(1,target_valuex, 2, target_valuey)
        
    # 이전에 생성한 카메라 프리뷰 위젯에 결과 이미지를 보임
    body_image.value = bgr8_to_jpeg(frame)
        
        
            
    # 결과 이미지를 프리뷰 위젯에 표시
    body_image.value = bgr8_to_jpeg(frame)

220
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\n\x00\n\x02\t4\x00\n\xdc'
32
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xf8\x00\n\x02\t\x03\x00\n '
72
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xf0\x00\n\x02\x08\xe4\x00\nH'
102
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xec\x00\n\x02\x08\xca\x00\nf'
118
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xeb\x00\n\x02\x08\xbb\x00\nv'
128
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xec\x00\n\x02\x08\xb0\x00\n\x80'
127
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xef\x00\n\x02\x08\xae\x00\n\x7f'
128
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xf2\x00\n\x02\x08\xaa\x00\n\x80'
124
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xf8\x00\n\x02\x08\xa8\x00\n|'
118
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x02\x00\n\x02\x08\xa3\x00\nv'
112
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\r\x00\n\x02\x08\x9e\x00\np'
99
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x1a\x00\n\x02\x08\x9e\x00\nc'
81
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08$\x00\n\x02\x08\xa6\x00\nQ'
47
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08*\x00\n\x02\x08\xc2\x00\n/'
6
b'\xff\xff\xfe\x0e\x83*\x04\

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-ee6583683eed>", line 7, in <module>
    bodies = body_cascade.detectMultiScale(gray)  # 이미지 내에서 얼굴을 검출
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-package